In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('spotify_millsongdata.csv')

In [3]:
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df=df.sample(10000).drop('link' ,axis=1).reset_index(drop=True)

Text PreProcessing

In [5]:
df['text']=df['text'].replace(r'^\w\s',' ').replace(r'\n',' ' ,regex=True)

In [6]:
import nltk
from nltk.stem.porter import PorterStemmer

In [7]:
stemmer = PorterStemmer()

In [8]:
def token(txt):
    token=nltk.word_tokenize(txt)
    a=[stemmer.stem(w)for w in token]
    return " ".join(a)

In [9]:
token("You are beautiful, beautifully")

'you are beauti , beauti'

In [10]:
df['text'].apply(lambda x:token(x))

0       make up my life killer in the night rattl my b...
1       toss into my mind , stir the calm you splash m...
2       move like a fist through traffic anger and no ...
3       lord , i give myself , i trust in you , mighti...
4       vers 1 : frank , out on the street . work late...
                              ...                        
9995    most of us are sad no one let it show i 've be...
9996    ooh , wahh ooh , wahh i love how your eye clos...
9997    look on down from the bridg there 's still fou...
9998    there 's a stormi vision star befor my eye i s...
9999    front room back room slide into tabl crowd int...
Name: text, Length: 10000, dtype: object

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
tfid=TfidfVectorizer(analyzer='word' ,stop_words='english')

In [13]:
matrix=tfid.fit_transform(df['text'])

In [14]:
similar=cosine_similarity(matrix)

In [15]:
similar[0]

array([1.        , 0.0015954 , 0.01592554, ..., 0.01369647, 0.00301159,
       0.00791705])

In [16]:
df.tail()

,artist,song,text
9995,Eagles,Most Of Us Are Sad,Most of us are sad \r No one lets it show \r...
9996,Mud,I Love How To Love Me,"Ooh, wahh \r Ooh, wahh \r \r I love how yo..."
9997,Mazzy Star,Look On Down From The Bridge,Look on down from the bridge \r There's still...
9998,Doobie Brothers,Under The Spell,There's a stormy vision \r Stars before my ey...
9999,Joni Mitchell,Tax Free,Front rooms \r Back rooms \r Slide into tabl...


In [18]:
df[df['song']=='Peace Of Mind'].index[0]

5418

Recommender Function

In [19]:
def recommender(song_name):
    idx = df[df['song']==song_name].index[0]
    distance=sorted(list(enumerate(similar[idx])) ,reverse=True, key = lambda x:x[1])
    song=[]
    for s_id in distance[1:21]:
        song.append(df.iloc[s_id[0]].song)
    return song

In [20]:
recommender("Peace Of Mind")

['Anything Is Possible',
 'None Of Us Are Free',
 'Got To Be Free',
 'All Things Are Possible',
 'Got To Be Free',
 'Free Like We Want 2 B',
 'I Want To Break Free',
 "I'm Free",
 'Middle Man',
 'Lose Again',
 'All In Your Mind',
 'Impossible',
 'Peace',
 "It's Love",
 'If You Were The Only Girl In The World',
 'Break Free',
 'Nightingale',
 'Intro',
 'Someday My Peace Will Come',
 "I Won't Mind"]

In [21]:
import pickle

In [22]:
pickle.dump(similar, open("similarity" , "wb"))

In [23]:
pickle.dump(df, open("df" , "wb"))